**Installing the Dependencies**

In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-zy4pw_hu/unsloth_3ba0033395f94ac4bbc41cdea69464eb
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-zy4pw_hu/unsloth_3ba0033395f94ac4bbc41cdea69464eb
  Resolved https://github.com/unslothai/unsloth.git to commit e4c8ceacb3fca634f78e662873a01c37678fcb3e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 111.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 7.9 MB/s eta 0:00:00
   ━━

In [ ]:
from unsloth import FastLanguageModel
import torch
import time
import csv
import pandas as pd
import random

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [ ]:
# Set random seed for reproducibility
random.seed(42)

**Data Manipulation**

In [ ]:
file_dir = '/content/Dataset.csv'
df = pd.read_csv(file_dir)
df.head()

,id,input,gt
0,4755,<s> package net . ggtools . grand . ui . graph...,"owner , final Point location )"
1,4644,<s> package net . bioclipse . opentox . ds . w...,> models ;
2,2276,<s> package com . asakusafw . yaess . core ; i...,= <NUM_LIT:0> ;
3,5322,<s> package org . rubypeople . rdt . internal ...,CallHierarchyMessages . SearchScopeActionGroup...
4,2861,<s> package com . asakusafw . compiler . flow ...,Ex1 > in1 ;


In [ ]:
# Get the number of rows in the dataset
num_rows = len(df)
print("The dataset has ", num_rows, " rows.")

The dataset has  3000  rows.


In [ ]:
df['input'][5]

'<s> package com . asakusafw . dmdl . thundergate ; import java . util . Collection ; import java . util . Set ; import java . util . regex . Pattern ; import com . asakusafw . utils . collections . Sets ; public interface ModelMatcher { ModelMatcher ALL = new ModelMatcher ( ) { @ Override public boolean acceptModel ( String name ) { return true ; } } ; ModelMatcher NOTHING = new ModelMatcher ( ) { @ Override public boolean acceptModel ( String name ) { return false ; } } ; boolean acceptModel ( String name ) ; public class And implements ModelMatcher { private final ModelMatcher [ ] matchers ; public And ( ModelMatcher ... matchers ) { if ( matchers == null ) { throw new IllegalArgumentException ( "<STR_LIT>" ) ; } this . matchers = matchers . clone ( ) ; } @ Override public boolean acceptModel ( String name ) { for ( ModelMatcher m : matchers ) { if ( m . acceptModel ( name ) == false ) { return false ; } } return true ; } } public class Not implements ModelMatcher { private final Mo

**Code Segments Cleaning**

In [ ]:
df['input'] = (
    df['input']
    .str.replace(r'<s> ', '', regex=True)
    .str.replace(r'\. ', '.', regex=True)
    .str.replace(r' ;', ';', regex=True)
    .str.replace(r' ,', ',', regex=True)
    .str.replace(r' \.', '.', regex=True)
    .str.replace(r' < ', '<', regex=True)
    .str.replace(r' >', '>', regex=True)
    .str.replace(r' \(', '(', regex=True)
    .str.replace(r' \)', ')', regex=True)
    .str.replace(r'\( ', '(', regex=True)
    .str.replace(r' \[', '[', regex=True)
    .str.replace(r' \]', ']', regex=True)
    .str.replace(r' \{', '{', regex=True)
    .str.replace(r' \}','}', regex=True)
    .str.replace(r'@ ','@', regex=True)
)

df.head()

,id,input,gt
0,4755,package net.ggtools.grand.ui.graph.draw2d; imp...,"owner , final Point location )"
1,4644,package net.bioclipse.opentox.ds.wizards; impo...,> models ;
2,2276,package com.asakusafw.yaess.core; import java....,= <NUM_LIT:0> ;
3,5322,package org.rubypeople.rdt.internal.ui.callhie...,CallHierarchyMessages . SearchScopeActionGroup...
4,2861,package com.asakusafw.compiler.flow.processor....,Ex1 > in1 ;


In [ ]:
df['input'][5]

'package com.asakusafw.dmdl.thundergate; import java.util.Collection; import java.util.Set; import java.util.regex.Pattern; import com.asakusafw.utils.collections.Sets; public interface ModelMatcher{ ModelMatcher ALL = new ModelMatcher(){ @Override public boolean acceptModel(String name){ return true;}}; ModelMatcher NOTHING = new ModelMatcher(){ @Override public boolean acceptModel(String name){ return false;}}; boolean acceptModel(String name); public class And implements ModelMatcher{ private final ModelMatcher[] matchers; public And(ModelMatcher...matchers){ if(matchers == null){ throw new IllegalArgumentException("<STR_LIT>");} this.matchers = matchers.clone();} @Override public boolean acceptModel(String name){ for(ModelMatcher m : matchers){ if(m.acceptModel(name) == false){ return false;}} return true;}} public class Not implements ModelMatcher{ private final ModelMatcher term; public Not(ModelMatcher term){ if(term == null){ throw new IllegalArgumentException("<STR_LIT>");} th

**Examples Selection**

In [ ]:
num_rows = len(df)
indices = random.sample(range(num_rows), 5)

examples = []
for idx in indices:
    example = df.iloc[idx]
    examples.append(example)

examples_df = pd.DataFrame(examples)

example1, example2, example3, example4, example5 = examples

df = df.drop(indices, axis=0).reset_index(drop=True)

In [ ]:
examples_df

,id,input,gt
2619,4112,package com.asakusafw.windgate.hadoopfs.ssh; i...,"byte [ ] b , int off , int len ) throws IOExce..."
456,4781,package com.lmax.disruptor.support; import jav...,class DaemonThreadFactory implements ThreadFac...
102,4938,package org.rubypeople.rdt.refactoring.ui.page...,public static String ParametersTableCellEditor...
1126,4137,package com.asakusafw.windgate.file.resource; ...,( ) throws Exception
1003,6642,package org.rubypeople.rdt.internal.debug.ui; ...,rdt . ui . viewsupport . ImageDescriptorRegist...


In [ ]:
examples_data_dir = '/content/Examples.csv'
examples_df.to_csv(examples_data_dir, index=False)

new_data_dir = '/content/Dataset_modified.csv'
df.to_csv(new_data_dir, index=False)

In [ ]:
example4['input']

'package com.asakusafw.windgate.file.resource; import static org.hamcrest.CoreMatchers.*; import static org.junit.Assert.*; import java.io.File; import java.io.FileOutputStream; import java.io.IOException; import java.io.ObjectOutputStream; import java.util.Arrays; import java.util.Collections; import org.junit.Rule; import org.junit.Test; import org.junit.rules.TemporaryFolder; import com.asakusafw.windgate.core.DriverScript; import com.asakusafw.windgate.core.GateScript; import com.asakusafw.windgate.core.ProcessScript; import com.asakusafw.windgate.core.resource.DrainDriver; import com.asakusafw.windgate.core.resource.SourceDriver; import com.asakusafw.windgate.core.vocabulary.FileProcess; public class FileResourceMirrorTest{ @Rule public TemporaryFolder folder = new TemporaryFolder(); @Test public void getName() throws Exception{ FileResourceMirror resource = new FileResourceMirror("<STR_LIT>"); try{ assertThat(resource.getName(), is("<STR_LIT>"));} finally{ resource.close();}} @Te

In [ ]:
example4['gt']

'( ) throws Exception'

In [ ]:
# Get the number of rows in the dataset
num_rows = len(df)
print("The dataset has ", num_rows, " rows.")

The dataset has  2995  rows.


**LLM Configuration**

In [ ]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-70B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.0.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/331k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/7.00G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/6.90G [00:00<?, ?B/s]

model-00003-of-00006.safetensors:   0%|          | 0.00/6.94G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

In [ ]:
FastLanguageModel.for_inference(model)

In [ ]:
print(next(model.parameters()).device)

cuda:0


In [ ]:
file_path = '/content/Dataset_modified.csv'
data = pd.read_csv(file_path)

data.head()

,id,input,gt
0,4755,package net.ggtools.grand.ui.graph.draw2d; imp...,"owner , final Point location )"
1,4644,package net.bioclipse.opentox.ds.wizards; impo...,> models ;
2,2276,package com.asakusafw.yaess.core; import java....,= <NUM_LIT:0> ;
3,5322,package org.rubypeople.rdt.internal.ui.callhie...,CallHierarchyMessages . SearchScopeActionGroup...
4,2861,package com.asakusafw.compiler.flow.processor....,Ex1 > in1 ;


In [ ]:
content = data['input'][1]
content

'package net.bioclipse.opentox.ds.wizards; import java.util.List; import java.util.Map; import net.bioclipse.opentox.ds.OpenToxModel; import org.eclipse.jface.wizard.IWizardPage; import org.eclipse.jface.wizard.Wizard; public class AddModelsWizard extends Wizard{ private SelectModelsPage selectModelsPage; private List<OpenToxModel'

**Templates for Chat Models**

In [ ]:
zero_shot_messages_template = [
    {
        "role": "system",
        "content": "You are an AI assistant specialized in code completion for Java. Your task is to complete the provided Java code segment with one line. Give only the code completion.",
    },
    {
        "role": "user",
        "content": None
    },
]

one_shot_messages_template = [
    {
        "role": "system",
        "content": "You are an AI assistant specialized in code completion for Java. Your task is to complete the provided Java code segment with one line. Give only the code completion.",
    },
    {
        "role": "user",
        "content": example1['input']
    },
    {
        "role": "assistant",
        "content": example1['gt']
    },
    {
        "role": "user",
        "content": None
    },
]

few_shot_messages_template = [
    {
        "role": "system",
        "content": "You are an AI assistant specialized in code completion for Java. Your task is to complete the provided Java code segment with one line. Give only the code completion.",
    },
    {
        "role": "user",
        "content": example1['input']
    },
    {
        "role": "assistant",
        "content": example1['gt']
    },
    {
        "role": "user",
        "content": example2['input']
    },
    {
        "role": "assistant",
        "content": example2['gt']
    },
    {
        "role": "user",
        "content": example3['input']
    },
    {
        "role": "assistant",
        "content": example3['gt']
    },
    {
        "role": "user",
        "content": example4['input']
    },
    {
        "role": "assistant",
        "content": example4['gt']
    },
    {
        "role": "user",
        "content": example5['input']
    },
    {
        "role": "assistant",
        "content": example5['gt']
    },
    {
        "role": "user",
        "content": None
    },
]

chain_of_thought_messages_template = [
    {
        "role": "system",
        "content": "You are an AI assistant specialized in code completion for Java. Your task is to complete the provided Java code segment with one line. Provide a detailed reasoning process before giving the final code completion.",
    },
    {
        "role": "user",
        "content": f"Here is the Java code segment: {example1['input']}. Reason through how to complete it and provide the final code completion.",
    },
    {
        "role": "assistant",
        "content": f"To complete the code segment, I need to consider the following points:\n\n1. **Analyze the Code Segment**: The code segment defines a class `ZipEntryInputStream` that extends `InputStream` and includes several methods related to zip file handling. The segment is incomplete and needs the implementation of the `read(byte[] b, int off, int len)` method.\n2. **Identify Missing Components**: The `read(byte[] b, int off, int len)` method is a standard method in `InputStream` for reading bytes from a stream with an offset and length. This method must be implemented to complete the class.\n3. **Determine the Code Addition**: Based on the `InputStream` class requirements and the provided context, I need to provide a method implementation that reads bytes into an array from a specific offset up to a length.\n\n**Final Code Completion**:\n{example1['gt']}",
    },
    {
        "role": "user",
        "content": None
    },
]


**Datasets Creation**

In [ ]:
# Function to generate dataset
def generate_dataset(messages_template, data, filename):
    chat_list = []
    input_ids_list = []
    num_tokens_list = []
    gt_list = []

    for _, row in data.iterrows():
        content = row['input']
        gt_value = row['gt']

        messages = [dict(message) for message in messages_template]
        messages[-1]['content'] = content

        input_ids = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to("cuda")

        chat_list.append(messages)
        input_ids_list.append(input_ids[0].cpu().numpy().tolist())
        num_tokens_list.append(len(input_ids[0]))
        gt_list.append(gt_value)

    results_df = pd.DataFrame({
        'Chat': chat_list,
        'input_ids': input_ids_list,
        'number_of_tokens': num_tokens_list,
        'gt': gt_list
    })

    results_df.to_csv(filename, index=False)

In [ ]:
# Generate datasets for each message template
generate_dataset(zero_shot_messages_template, data, '/content/Zero_shot_Dataset.csv')
generate_dataset(one_shot_messages_template, data, '/content/One_shot_Dataset.csv')
generate_dataset(few_shot_messages_template, data, '/content/Few_shot_Dataset.csv')
generate_dataset(chain_of_thought_messages_template, data, '/content/Chain_of_thought_Dataset.csv')